In [3]:
#unzippinf lists_to_use zip to access files inside it
from zipfile import ZipFile

with ZipFile('../intermediate/lists_to_use.zip', 'r') as f:
    f.extractall('../intermediate/')

#accessing files
import pickle
with open('../intermediate/lists_to_use/jl.pickle', 'rb') as file:
  jgslist = pickle.load(file)

with open('../intermediate/lists_to_use/sl.pickle', 'rb') as file:
  sumlist = pickle.load(file)

#remove unzipped files
import os
os.system("rm -r ../intermediate/lists_to_use")

0

1. jgslist and sumlist is a list of list of sentences that is preprocessed
2. jgslist[i] is a list that contains sentences of ith judgement
3. sumlist[i] is a list that contains sentences of ith judgement's summary
4. len(jgslist) = len(sumlist) = 7028 (we have 7028 judgements in our dataset)
5. stored jgslist and sumlist in preprocessing.ipynb file present in the root directory

Found that lemmatizing our words may reduce chances of OOV words problem

Lemmatized each word in our dataset using nltk WordNetLemmatizer

In [4]:
import numpy as np
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

In [4]:
import torch  
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")
model = AutoModel.from_pretrained("nlpaueb/legal-bert-base-uncased")

Some weights of the model checkpoint at nlpaueb/legal-bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
#takes list of strings as a parameter
#return list where each element 

def get_sentence_embeddings(sentences):
    # Tokenize the sentences
    encoded_inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

    # Forward pass through the BERT model
    with torch.no_grad():
        outputs = model(**encoded_inputs)
        sentence_embeddings = outputs.last_hidden_state.mean(dim=1)  # Average pooling

    return sentence_embeddings


In [22]:
#just checking whether each sentence embedding is correctly converted to list or some other datatype

temp = ["paper publish","of course"]
tvec = get_sentence_embeddings(temp)
for vect in tvec:
    print(vect.shape)

torch.Size([768])
torch.Size([768])


Finding vector of each sentences in jgslist and sumlist and storing it in list

In [ ]:
#finding sentence embeddings for each sentence

#lst is a list of sentences of a particular judgement
jgsvector = [get_sentence_embeddings(lst) for lst in jgslist]

#lst is a list of sentences of a particular judgement
sumvector = [get_sentence_embeddings(lst) for lst in sumlist]

In [ ]:
# for a particular vector is jgsvector there will be a similarity score
# so for vectors in jgsvector[i] we have to compare each vector with vectors in sumvector[i] and add the scores for that vector in jgsvector[i]
scores = []
for i in range(len(jgsvector)):
  temp = []
  for j in range(len(jgsvector[i])):
    #compare jth sentence with all sentences in sumlist[i] and add scores
    score = 0
    vect1 = np.array(jgsvector[i][j])
    for lst in sumvector[i]:
      vect2 = np.array(lst)
      score += cosine(vect1,vect2)
    temp.append(score)
  scores.append(temp)

In [ ]:
#store my scores list, sentence emebeddings in intermediate folder                      
# Save data to a pickle file
                
with open('../intermediate/scores_legbert.pickle', 'wb') as file:
    pickle.dump(scores, file)

with open('../intermediate/jgsvector_legbert.pickle', 'wb') as file:
    pickle.dump(jgsvector, file)

with open('../intermediate/sumvector_legbert.pickle', 'wb') as file:
    pickle.dump(sumvector, file)


import os
os.system("zip ../intermediate/legbert.zip ../intermediate/scores_legbert.pickle ../intermediate/jgsvector_legbert.pickle ../intermediate/sumvector_legbert.pickle")
os.system("rm ../intermediate/scores_legbert.pickle")
os.system("rm ../intermediate/jgsvector_legbert.pickle")
os.system("rm ../intermediate/sumvector_legbert.pickle")
